#### Project: A Vanilla Generative Adversarial Networks

#### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [5]:
import os
import torch
from torchvision import datasets
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

#### Settings
- Configure the device
- define all the hyperparameters to be used and needed to be tuned to achive a better accuracy
- Load and explore the data

In [9]:
# device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
learning_rate = 0.002
sample_dir = 'samples'

# Create directory id not exits
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    
# Image preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],
                        std=[0.5])])

# dataset => MNIST
train_dataset = datasets.MNIST(root='data',
                              train=True,
                               transform=transform,
                               download=True)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)



#### Define the architecture of the model such as
- Define the Generator architecture
- Define the Critic/Discriminator architecture

In [10]:
# Discriminator
Discriminator = nn.Sequential(
                nn.Linear(image_size, hidden_size),
                nn.LeakyReLU(0.2),
                nn.Linear(hidden_size, hidden_size),
                nn.LeakyReLU(0.2),
                nn.Linear(hidden_size, 1),
                nn.Sigmoid())

# Generator
Generator = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, image_size),
            nn.Tanh())


#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum etc.

In [13]:
Generator = Generator.to(device)
Discriminator = Discriminator.to(device)

criterion = nn.BCELoss()
# optimizer for the Generator
g_optimizer = torch.optim.Adam(Generator.parameters(), lr=learning_rate)
d_optimizer = torch.optim.Adam(Discriminator.parameters(), lr=learning_rate)

In [14]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

#### Training a model requires the following steps
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)